# SETUP

In [9]:
import sys
import random
import numpy as np
import pandas as pd
import pymorphy2
from nltk.corpus import stopwords
import joblib
import nltk
nltk.download("stopwords")

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from keras.layers import LeakyReLU

import warnings
warnings.filterwarnings('ignore')

In [ ]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
path_rep = !pwd
path_rep = path_rep[0]

# установим путь к модулям
sys.path.append(f'{path_rep}/modules')

from TabularDataAnalysis import primary_info_cols

# преобразовать табличные данные после EDA
from TabularDataTransform import transform_tabular_base_data
# feauture enginering после первичного преобразования
from TabularDataTransform import feauture_enginering_tab_data
# формировать датафрейм
from TabularDataTransform import create_df

# грфик ошибки по эпохам
from Metrics import history_loss_metrics
# MAPE
from Metrics import mape

# создать датафрейм для единого преобразования текстов
from NLPDataTransform import create_nlp_df
# перевести тексты в нижний регистр, очистить от символов вне паттерна, применить лемматизацию, удалить стоп-слова
from NLPDataTransform import clear_nlp_data
# Токенизация "Мешок слов"
from NLPDataTransform import bag_of_words_tokenize
# Токенизация по словосочетанию из 2-х слов
from NLPDataTransform import two_gram_tokenize

# вывести изображения и стоимость
from ImageDataAnalysis import img_price_show
# вывести пример аугментации
from ImageDataAnalysis import view_sample_augmentation

# векторизовать изображения
from ImageTransform import get_image_array
# настройки для аугментации
from ImageTransform import custom_augmentation

# DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

# Tabular

Работаем с табличными данными

## Подготовим табличные данные

In [ ]:
# загрузим данные
DATA_DIR = 'data'
train = pd.read_csv(DATA_DIR + '/train.csv')
test = pd.read_csv(DATA_DIR + '/test.csv')
sample_submission = pd.read_csv(DATA_DIR + '/sample_submission.csv')

In [ ]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

## EDA

### Посмотрим на каждый признак и определим преобразования

In [ ]:
primary_info_cols(data=data,cols=data.columns[:-2])

### Внесем преобразования

In [ ]:
transform_tabular_base_data(data=data)

### EDA результат

- bodyType - Убрал информацию о дверях (тк есть отдельный признак), объединил редкие данные с похожими основными типами
- description - не подходит для табличной модели, но множно вытащить число символов описания
- engineDisplacement - перевел в числовой формат, установил mean для undefined
- enginePower - перевел в числовой формат
- **model_info - возможно, следует объединить brand и info в один признак, чтобы info не путались для разных брендов**
- **name - Выделить отдельным признаком long, compact, competition, xDrive, AMG, Blue. Потом удалить, тк дублирует признаки**
- **vehicleConfiguration - дублирует - удалить**
- **Владение - переведел в числовой формат (float) в годы, установил для nan среднее занчение в зависимости от ModelDate. Но проверить, не влияет ли негативно большое количество nan для точности**
- **Руль - недостаточно данных - удалить**

## Feauture Enginering

In [ ]:
feauture_enginering_tab_data(data=data)

- long, compact, competition, xDrive, AMG, Blue - характеристики из name
- description_len - количество символов в описании
- productionDate_minus_modelDate - разница даты производства и даты выхода моделей
- enginePower_on_engineDisplacement - мощность двигателя на объем
- mileage_on_enginePower - пробег на мощность двигателя
- mileage_on_engineDisplacement - пробег на объем двигателя
- mileage_on_Владение - пробег на срок владения автомобилем
- productionDate_max_minus_modelDate - лет с момента выпуска модели
- productionDate_max_minus_productionDate - лет с момента производства автомобиля
- key_words_description - ключевые слова из описания объявления, влияющие на стоимость
- mileage_on_date - миль на возраст автомобиля

## Анализ корреляций признаков

In [ ]:
# оставим только тренировочные данные с прайсом
train_data_analytics = data.query('sample == 1').drop(['sample'], axis=1)

In [ ]:
# корреляция
fig, ax = plt.subplots(1, 1, figsize=(30, 15))
ax = sns.heatmap(train_data_analytics.corr(),fmt='.1g',
                 annot=True, cmap='coolwarm')

- engineDisplacement и enginePower 0.9, enginePower имеет большую корреляцию с таргет. Оставляем оба, тк высокая корреляция с таргетом.
- mileage и modelDate/productionDate -0.7, mileage_on_enginePower/mileage_on_engineDisplacement 0.9, mileage_on_productionDate_norm100 1, productionDate_max_minus_modelDate/productionDate_max_minus_productionDate 0,7. **mileage_on_productionDate_norm100, mileage_on_enginePower - удалить**
- modelDate и productionDate 1, productionDate_max_minus_modelDate/productionDate_max_minus_productionDate -1. **modelDate, productionDate, productionDate_max_minus_productionDate - удалить**

## Сформируем датафрейм

In [ ]:
df = create_df(data=data)

In [ ]:
# разделим на тренировочные и тестовые данные
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

In [ ]:
# обучающие и тестовые данные
y = train_data.price.values
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

## Tabular Models

### Neural Networks

Лучшим mlp является NN 1 (Simple), TEST mape: 10.55%

#### Neural Network 1 (Simple)

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    'models/tabular/nn1.hdf5', monitor='val_MAPE',
                    save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,) 
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=350, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict(X_test)[:,0]
mape(y_true=y_test, y_pred=y_pred)

TEST mape: 10.55%

#### Neural Network 2 (Relu to LeakyRelu)

In [ ]:
model = Sequential()
# model.add(L.Dense(1024, input_dim=X_train.shape[1], activation="relu"))
# model.add(L.Dropout(0.5))
model.add(L.Dense(512, input_dim=X_train.shape[1]))
model.add(LeakyReLU())
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dense(256, activation="relu")) #добавили допонительный полносвязный слой
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    'models/tabular/nn2.hdf5', monitor='val_MAPE',
                    save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,) 
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=350, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict(X_test)[:,0]
mape(y_true=y_test, y_pred=y_pred)

full LeakyRelu: 11.07%

first LeakyRelu: 10.62%

#### Neural Network 3 (bottle neck)

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(128, activation="relu"))
model.add(L.Dropout(0.25))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(512, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    'models/tabular/nn3.hdf5', monitor='val_MAPE',
                    save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,) 
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=350, # обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=1,
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict(X_test)[:,0]
mape(y_true=y_test, y_pred=y_pred)

mape: 10.81%

### CatBoost

Отдельно проведя gridsearch не добился лучшего результата, используем параметры по умолчанию

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                         )
model.fit(X_train, np.log(y_train), # логарифм
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=100,
         use_best_model=True,
         )

In [ ]:
# метрика
y_pred = np.exp(model.predict(X_test))
mape(y_true=y_test, y_pred=y_pred)

TEST mape: 11.17%

In [ ]:
# сохраним модель
joblib.dump(model, f'models/tabular/catboost.pkl')

In [ ]:
# сделаем предсказание и сохраним результат
sub_predict_catboost = np.exp(model.predict(X_sub))
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('subs/catboost_submission.csv', index=False)

### Neural Network 1 (Simple) - optimize

Попробуем улучшить наиболее качественную сеть (NN 1 Simple), снизив шаг спуска

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
# Загрузим веса
model.load_weights('models/tabular/nn1.hdf5')

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(ExponentialDecay(1e-2, 100, 0.9))
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    'models/tabular/tabular_best_model.hdf5', monitor='val_MAPE',
                    save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=150, restore_best_weights=True,) 
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=1000, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=1,
                   )

In [ ]:
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict(X_test)[:,0]
mape(y_true=y_test, y_pred=y_pred)


best_model_finish: 10.44%

### Результат

<table>
  <tbody><tr>
    <th>Модель</th>
    <th>Описание</th>
    <th>best MAPE</th>
    <th>Комментарий</th>
  </tr>
    <tr>
    <td>Neural Network 1 (Simple)</td>
    <td>Базовая сеть - классический персептрон</td>
    <td>10.55%</td>
    <td>Лучший результат</td>
  </tr>
  <tr>
    <td>Neural Network 2 (Relu to LeakyRelu)</td>
    <td>Снизу используем слой LeakyReLU</td>
    <td>10.62%</td>
    <td>Резульат хуже базового</td>
  </tr>
  <tr>
    <td>Neural Network 3 (bottle neck)</td>
    <td>Сеть по принципу bottle neck</td>
    <td>10.81%</td>
    <td>Резульат хуже базового</td>
  </tr>
  <tr>
    <td>CatBoost</td>
    <td>Подбор гиперпараметров с помощью gridsearch не помог, логарифмирование y улучшило метрику</td>
    <td>11.17%</td>
    <td>Достигли лучшего результата для CatBoost</td>
  </tr>
  <tr>
    <td>Neural Network 1 (Simple) - optimize</td>
    <td>Используем веса базовой сети и переобучим, снизив шаг спуска</td>
    <td>10.44%</td>
    <td>Достигли лучшего результата</td>
  </tr>
</tbody></table>

# Tabular + NLP

Создадим модель с подачей на вход табличных и текстовых данных

## NLP Data

In [ ]:
# объединим тексты трейна и теста для преобразования
data_nlp = create_nlp_df(train, test)

## Очистка данных

In [ ]:
# составим паттерны для чисел и русских и английских символов
patterns = "[^A-Za-z0-9А-Яа-я]+"
# добавим лемматизацию
morph = pymorphy2.MorphAnalyzer()
# составим список стоп-слов
russian_stopwords = stopwords.words("russian")

In [ ]:
# перевести тексты в нижний регистр, очистить от символов вне паттерна, применить лемматизацию, удалить стоп-слова
data_nlp = clear_nlp_data(data_nlp, patterns, morph, russian_stopwords)

## Подготовим данные на вход модели

In [ ]:
# разделим трейн, валидационные и тест
text_train = data_nlp.description.iloc[X_train.index]
text_test = data_nlp.description.iloc[X_test.index]
text_sub = data_nlp.description.iloc[X_sub.index]

Протестировав мешок слов и 2 gram на простой сети определил, что с мешком качество чуть лучше (см. ниже)

### Bag of words

In [ ]:
text_train_sequences, text_test_sequences, text_sub_sequences, tokenize = bag_of_words_tokenize(
    text_train, text_test, text_sub, all_texts=data_nlp['description'])

### 2 gram

In [ ]:
text_train_sequences_2, text_test_sequences_2, text_sub_sequences_2 = two_gram_tokenize(
    X_train.index, X_test.index, X_sub.index, all_texts=data_nlp['description'])

## Обучим модель Tabular + Text

In [ ]:
# установим длинну вектора
MAX_SEQUENCE_LENGTH = 256

### Multi-Input: Base MLP+NLP (bag of words) (best model)

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(64, activation="sigmoid")) #добавим полносвязный слой
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

In [ ]:
# объединим слои в голове
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('models/mlp_nlp/nlp_mlp_base_bow.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict([text_test_sequences, X_test])[:,0]
mape(y_true=y_test, y_pred=y_pred)

Уменьшим шаг

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('models/mlp_nlp/nlp_mlp_base_bow.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=25, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
# метрика
y_pred = model.predict([text_test_sequences, X_test])[:,0]
mape(y_true=y_test, y_pred=y_pred)

TEST mape (с числовыми символами): 10.71%

TEST mape (только русские и английские символы): 10.73%

TEST mape (только русские): 10.96%

TEST mape (с числами и русскими и английскими символами): 10.64%

### Base MLP+NLP (2 gram)

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=512, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(64, activation="sigmoid")) #добавим полносвязный слой
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

In [ ]:
# объединим слои в голове
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model_2 = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model_2.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('models/mlp_nlp/nlp_mlp_base_2_gram.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model_2.fit([text_train_sequences_2, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences_2, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict([text_test_sequences_2, X_test])[:,0]
mape(y_true=y_test, y_pred=y_pred)

TEST mape: 10.72%

**bag of words показывает лучший результат - оставляем метод**

### Test model v2

С полносвязными слоями с большим числом нейронов ближе к голове

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
# # https://www.kaggle.com/code/arunrk7/nlp-beginner-text-classification-using-lstm
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.SpatialDropout1D(0.2))
model_nlp.add(L.Conv1D(64, 5, activation='relu'))
model_nlp.add(L.Bidirectional(L.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model_nlp.add(L.Dense(512, activation='relu'))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(512, activation='relu'))

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('models/mlp_nlp/nlp_mlp_2.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict([text_test_sequences, X_test])[:,0]
mape(y_true=y_test, y_pred=y_pred)

TEST mape: 11.26%

### Test model v3

Мало слоев, без LSTM

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.GlobalMaxPool1D())
model_nlp.add(L.Dense(10, activation='relu'))

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
# объединим слои в голове
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('models/mlp_nlp/nlp_mlp_3.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict([text_test_sequences, X_test])[:,0]
mape(y_true=y_test, y_pred=y_pred)

TEST mape: 11.58%

### Результаты тестирования архитектур

<table>
  <tbody><tr>
    <th>Модель</th>
    <th>Описание</th>
    <th>best MAPE</th>
    <th>Комментарий</th>
  </tr>
  <tr>
    <td>Base</td>
    <td>Постепенно снижение количества нейронов в слоях ближе к голове, с полносвязным слоем по середине и dropuot после слоев нейронов</td>
    <td>10.64%</td>
    <td>Лучший результат</td>
  </tr>
    <tr>
    <td>Test model v2</td>
    <td>С полносвязными слоями с большим числом нейронов ближе к голове</td>
    <td>11.26%</td>
    <td>Результат хуже базовой</td>
  </tr>
  <tr>
    <td>Test model v3</td>
    <td>Мало слоев, без LSTM</td>
    <td>11.58%</td>
    <td>Худший результат</td>
  </tr>
</tbody></table>

**Базовая архитектура показывает лучший результат**

# MLP+NLP+CV

Создадим модель с подачей на вход табличных, текстовых и графических данных

## Подготовка изображений

In [ ]:
# убедимся, что цены и фото подгрузились верно
img_price_show(data=train, data_dir=DATA_DIR)

In [ ]:
# векторизуем изображения
images_train = get_image_array(data=data, index=X_train.index, data_dir=DATA_DIR)
images_test = get_image_array(data=data, index=X_test.index, data_dir=DATA_DIR)
images_sub = get_image_array(data=data, index=X_sub.index, data_dir=DATA_DIR)

In [ ]:
# сохраним настройки для аугментации
AUGMENTATIONS = custom_augmentation()

In [ ]:
view_sample_augmentation(image=images_train[0], AUGMENTATIONS=AUGMENTATIONS)

## Подготовка данных на вход модели

Создим функции для формирования партий табличных и текстовых данных, матриц изображений для входа в комбинированную сеть

In [ ]:
# применить аугментацию к изображениям
def process_image(image):
    return AUGMENTATIONS(image = image.numpy())['image']

# векторизация текста
def tokenize_text(text):
    return sequence.pad_sequences(tokenize.texts_to_sequences([text.numpy().decode('utf-8')]), maxlen = MAX_SEQUENCE_LENGTH)[0]

# Подготовить партии данных (табличные, текстовые, графические)
def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

# Подготовить партии данных (табличные, графические)
def tf_process_train_dataset_element_tabular_image(image, table_data, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    return (image, table_data), price

def tf_process_val_dataset_element_tabular_image(image, table_data, price):
    return (image, table_data), price

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

## Построим модель (EfficientNetB3) (best)

In [ ]:
# размер изображений
size = (320, 240)

In [ ]:
# загрузим предобученную модель
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))

In [ ]:
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
# откроем все слои для обучения
efficientnet_model.trainable = True

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(64, activation="sigmoid")) #добавим полносвязный слой
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

In [ ]:
# объединим слои в голове
combinedInput = L.concatenate([efficientnet_output, model_mlp.output, model_nlp.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, model_mlp.input, model_nlp.input], outputs=head)

In [ ]:
# уменьшим шаг
optimizer = tf.keras.optimizers.Adam(ExponentialDecay(1e-3, 100, 0.9)) # 1e-3
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('models/mlp_nlp_cv/cv+nlp+mlp.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict(test_dataset.batch(30))[:,0]
mape(y_true=y_test, y_pred=y_pred)

TEST mape: 11.06%

## Протестируем модель mlp+cv

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, y_train
    )).map(tf_process_train_dataset_element_tabular_image)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, y_test
    )).map(tf_process_val_dataset_element_tabular_image)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, y_sub
    )).map(tf_process_val_dataset_element_tabular_image)

In [ ]:
# загрузим предобученную модель
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))

In [ ]:
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
# откроем все слои для обучения
efficientnet_model.trainable = True

In [ ]:
# заморозим первую половину
fine_tune_at = len(efficientnet_model.layers)//2

# Keep all other layers frozen
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
# объединим слои в голове
combinedInput = L.concatenate([efficientnet_output, model_mlp.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, model_mlp.input], outputs=head)

In [ ]:
optimizer = tf.keras.optimizers.Adam(ExponentialDecay(1e-3, 100, 0.9))
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('models/mlp_cv/mlp_cvp.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
# история обучения
history_loss_metrics(history)

In [ ]:
# метрика
y_pred = model.predict(test_dataset.batch(30))[:,0]
mape(y_true=y_test, y_pred=y_pred)

TEST mape: 11.16%

# Сделаем предсказание для соревнования

sub = sum(pred(i) * W(i))

## Only MLP

Только табличные данные

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
# загрузим веса
model.load_weights('models/tabular/tabular_best_model.hdf5')

In [ ]:
predict_mlp = model.predict(X_sub)

In [ ]:
sample_submission['price'] = predict_mlp

In [ ]:
sample_submission.to_csv('subs/mlp_submission.csv', index=False)

score: 11.00814

## MLP(0.5) + MLP+NLP(0.3) + MLP+CV(0.2)

### MLP_NLP

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(64, activation="sigmoid")) #добавим полносвязный слой
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
# Загрузим веса
model.load_weights('models/mlp_nlp/nlp_mlp_base_bow.hdf5')

In [ ]:
# сохраним резульатат для объединения
predict_mlp_nlp = model.predict([text_sub_sequences, X_sub])

### MLP+CV

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, y_train
    )).map(tf_process_train_dataset_element_tabular_image)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, y_test
    )).map(tf_process_val_dataset_element_tabular_image)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, y_sub
    )).map(tf_process_val_dataset_element_tabular_image)

In [ ]:
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = None, include_top = False, input_shape = (size[1], size[0], 3))

In [ ]:
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
combinedInput = L.concatenate([efficientnet_output, model_mlp.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, model_mlp.input], outputs=head)

In [ ]:
# Загрузим веса
model.load_weights('models/mlp_cv/mlp_cvp.hdf5')

In [ ]:
# сохраним для объединения
predict_mlp_cv = model.predict(sub_dataset.batch(30))

### Объединяю предикты

In [ ]:
# распределяю веса для рещультатов в зависимости от качества на валидации
blend_sub_predict = ((predict_mlp*0.5) + (predict_mlp_nlp*0.3) + (predict_mlp_cv*0.2))
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('subs/mlp50_mlpnlp25_mlpcv25submission.csv', index=False)

score: 10.98914

## MLP(0.4) + MLP+NLP(0.2) + MLP+CV(0.2) + MLP+NLP+CV(0.2)

### MLP+NLP+CV

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

In [ ]:
combinedInput = L.concatenate([efficientnet_output, model_mlp.output, model_nlp.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, model_mlp.input, model_nlp.input], outputs=head)

In [ ]:
# Загрузим веса
model.load_weights('models/mlp_nlp_cv/cv+nlp+mlp.hdf5')

In [ ]:
# сохраним для объединения
predict_mlp_nlp_cv = model.predict(sub_dataset.batch(30))

### Объединяю предикты

In [ ]:
blend_sub_predict = ((predict_mlp*0.4) + (predict_mlp_nlp*0.2) + (predict_mlp_cv*0.2) + (predict_mlp_nlp_cv*0.2))
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('subs/mlp40_mlpnlp20_mlpcv20_mlpnlpcv20_submission.csv', index=False)

score: 11.09280

## CatB(0.5) + (MLP(0.5) + MLP+NLP(0.3) + MLP+CV(0.2)) || sum / 2

Объединим лучший результат с catboost

In [ ]:
# загрузим сабмит с лучшим результатом
sub2 = pd.read_csv('subs/mlp50_mlpnlp25_mlpcv25submission.csv')

In [ ]:
# загрузим лучший результат для catboost
sub_predict_catboost = pd.read_csv('subs/catboost_submission.csv')

In [ ]:
blend_sub_predict = (sub_predict_catboost + np.array(sub2['price'])) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('subs/catb50_(mlp50_mlpnlp25_mlpcv25)_per2_submission.csv', index=False)

**score: 10.84581 (best result, top 12) - достигли лучшего результата**